# 랩 5: 고객 대면 프론트엔드 애플리케이션 구축

## 개요

이전 랩들에서 메모리, 공유 도구, 프로덕션급 배포를 갖춘 포괄적인 고객 지원 에이전트를 구축했습니다. 이를 통해 에이전트 사용 사례를 프로토타입에서 프로덕션으로 이동시키는 AgentCore 서비스의 기능을 보여주었습니다. 이제 모든 애플리케이션에서 에이전트 런타임을 호출할 수 있습니다. 실제 애플리케이션에서는 고객이 사용할 수 있는 사용자 인터페이스가 있어야 합니다. 이제 고객이 실제로 에이전트와 상호작용할 수 있는 사용자 친화적인 프론트엔드를 만들 시간입니다.

**워크숍 여정:**
- **랩 1 (완료)**: 에이전트 프로토타입 생성 - 기능적인 고객 지원 에이전트 구축
- **랩 2 (완료)**: 메모리로 향상 - 대화 컨텍스트와 개인화 추가
- **랩 3 (완료)**: 게이트웨이 및 아이덴티티로 확장 - 에이전트 간 도구 안전하게 공유
- **랩 4 (완료)**: 프로덕션 배포 - 관찰 가능성을 갖춘 AgentCore 런타임 사용
- **랩 5 (현재)**: 사용자 인터페이스 구축 - 고객 대면 애플리케이션 생성

이 랩에서는 배포된 고객 지원 에이전트와 상호작용할 수 있는 직관적인 채팅 인터페이스를 고객에게 제공하는 **Streamlit 기반 웹 애플리케이션**을 만들겠습니다. 프론트엔드에는 다음이 포함됩니다:

- **보안 인증** - Amazon Cognito를 통한 사용자 로그인
- **실시간 채팅 인터페이스** - Streamlit 기반 대화형 UI
- **스트리밍 응답** - 더 나은 사용자 경험을 위한 라이브 응답 스트리밍
- **세션 관리** - 메모리를 통한 지속적인 대화
- **응답 시간** - 투명성을 위한 성능 메트릭

### 아키텍처 for Lab 5

우리의 프론트엔드 애플리케이션은 랩 4에서 배포한 AgentCore Runtime 엔드포인트에 연결하여 완전한 엔드투엔드 고객 지원 솔루션을 제공합니다:

<div style="text-align:left">
    <img src="images/architecture_lab5_streamlit.png" width="100%"/>
</div>


### 학습 내용

- 프론트엔드와 보안 인증을 통합하는 방법
- 실시간 스트리밍 응답을 구현하는 방법
- 사용자 세션과 대화 컨텍스트를 관리하는 방법
- 고객 지원을 위한 직관적인 채팅 인터페이스를 생성하는 방법

### 랩 목표

이 랩을 완료하면 다음을 갖게 됩니다:

- 고객 대면 Streamlit 웹 애플리케이션 배포
- AgentCore Identity와 보안 사용자 인증 통합
- 실시간 스트리밍 채팅 응답 구현
- 완전한 엔드투엔드 고객 지원 솔루션 생성
- 로그인부터 지원 해결까지 전체 고객 여정 테스트

## 전제조건

- **랩 1-4 완료**
- **Python 3.10+** 로컬 설치
- **Streamlit** 및 필요한 프론트엔드 종속성
- **AgentCore 런타임 엔드포인트** 랩 4에서 (배포되고 준비됨)
- **Amazon Cognito** 인증용으로 구성된 사용자 풀

### Step 1: 프론트엔드 종속성 설치

먼저 Streamlit 프론트엔드 애플리케이션에 필요한 패키지를 설치합니다.

In [ ]:
# Install frontend-specific dependencies
%pip install -r lab_helpers/lab5_frontend/requirements.txt -q
print("✅ Frontend dependencies installed successfully!")

### Step 2: 프론트엔드 아키텍처 이해

우리의 Streamlit 애플리케이션은 여러 핵심 구성 요소로 구성됩니다:

#### 핵심 구성 요소:

1. **main.py** - UI와 인증을 포함한 메인 Streamlit 애플리케이션
2. **chat.py** - 채팅 관리 및 AgentCore 런타임 통합
3. **chat_utils.py** - 메시지 포맷팅 및 표시를 위한 유틸리티 함수
4. **sagemaker_helper.py** - 접근 가능한 URL 생성을 위한 헬퍼

#### 인증 플로우:

1. 사용자가 Streamlit 애플리케이션에 접근
2. Amazon Cognito가 사용자 인증 처리
3. 유효한 JWT 토큰이 AgentCore 런타임 요청 인가에 사용됨
4. 사용자가 고객 지원 에이전트와 안전하게 상호작용 가능

### Step 3: 고객 지원 프론트엔드 실행 🚀

이제 Streamlit 애플리케이션을 시작합니다. 애플리케이션은 다음을 수행합니다:

1. **애플리케이션용 접근 가능한 URL 생성**
2. **포트 8501에서 Streamlit 서버 시작**
3. **랩 4에서 배포된 AgentCore 런타임에 연결**
4. **완전한 고객 지원 인터페이스 제공**

**중요 사항:**
- 애플리케이션은 중지할 때까지 계속 실행됩니다 (Ctrl+C)
- 랩 4의 AgentCore 런타임이 여전히 배포되고 실행 중인지 확인하세요
- Cognito 인증 토큰은 2시간 동안 유효합니다

In [ ]:
# Get the accessible URL for the Streamlit application
from lab_helpers.lab5_frontend.sagemaker_helper import get_streamlit_url

streamlit_url = get_streamlit_url()
print(f'\n🚀 Customer Support Streamlit Application URL:\n{streamlit_url}\n')

# Start the Streamlit application
!cd lab_helpers/lab5_frontend/ && streamlit run main.py

### Step 4: 고객 지원 애플리케이션 테스트

Streamlit 애플리케이션이 실행되면 완전한 고객 지원 경험을 테스트할 수 있습니다:

#### 인증 테스트:
1. **위에 제공된 고객 지원 Streamlit 애플리케이션 URL을 사용하여 애플리케이션에 접근**
2. **출력에 제공된 테스트 자격 증명으로 로그인**
3. **사용자 이름과 함께 환영 메시지가 표시되는지 확인**

<div style="text-align:left">
    <img src="images/lab5_streamlit_login.png"/>
</div>
<div>
    <img src="images/lab5_welcome_user.png"/>
</div>    


#### 테스트할 고객 지원 시나리오:

제품 정보 문의: "노트북의 사양은 무엇인가요?"

반품 정책 질문: "전자제품의 반품 정책은 무엇인가요?"

문제 해결 지원: "제 iPhone이 과열되고 있는데, 어떻게 해야 하나요?"

<div style="text-align:left">    
    <img src="images/lab5_agent_question.png" width="75%"/>
</div>

메모리 및 개인화 테스트: 대화를 나눈 후 페이지를 새로고침

<div style="text-align:left">
    <img src="images/lab5_agent_chat_history.png" width="75%"/>
</div>

#### 관찰할 사항:

- **실시간 스트리밍** - 응답이 생성되는 대로 나타남
- **응답 시간** - 각 응답과 함께 성능 메트릭 표시
- **메모리 지속성** - 에이전트가 대화 컨텍스트를 기억
- **도구 통합** - 에이전트가 다양한 쿼리에 적절한 도구 사용
- **전문적인 UI** - 깔끔하고 직관적인 고객 지원 인터페이스
- **오류 처리** - 모든 문제의 우아한 처리

## 🎉 랩 5 완료!

축하합니다! AI 기반 고객 지원 에이전트를 위한 완전한 고객 대면 프론트엔드 애플리케이션을 성공적으로 구축하고 배포했습니다. 달성한 내용은 다음과 같습니다:

### 구축한 것

- **웹 인터페이스** - Streamlit 기반 고객 지원 애플리케이션
- **보안 인증** - 사용자 관리를 위한 Amazon Cognito 통합
- **실시간 스트리밍** - 더 나은 사용자 경험을 위한 라이브 응답 스트리밍
- **세션 관리** - 상호작용 간 메모리를 통한 지속적인 대화
- **완전한 통합** - AgentCore 런타임에 연결된 프론트엔드

### 엔드투엔드 고객 지원 솔루션

이제 다음을 포함하는 **완전한 고객 지원 시스템**을 갖게 되었습니다:

1. **지능형 에이전트** (랩 1) - 사용자 정의 도구를 갖춘 AI 기반 지원
2. **지속적인 메모리** (랩 2) - 대화 컨텍스트 및 개인화
3. **공유 도구 및 아이덴티티** (랩 3) - 확장 가능한 도구 공유 및 접근 제어
4. **프로덕션 런타임** (랩 4) - 관찰 가능성을 갖춘 안전하고 확장 가능한 배포
5. **고객 프론트엔드** (랩 5) - 최종 사용자를 위한 웹 인터페이스

### 시연된 주요 기능

- **다중 턴 대화** - 에이전트가 상호작용 간 컨텍스트 유지
- **도구 통합** - 제품 정보, 반품 정책, 웹 검색의 원활한 사용
- **메모리 지속성** - 고객 선호도 및 이력 유지
- **실시간 성능** - 성능 메트릭을 포함한 스트리밍 응답
- **보안 및 아이덴티티** - 적절한 인증 및 인가
- **관찰 가능성** - 에이전트 동작의 완전한 추적 및 모니터링

### 다음 단계

고객 지원 솔루션을 더욱 향상시키려면 다음을 고려하세요:

- **사용자 정의 스타일링** - 회사의 디자인 시스템으로 프론트엔드 브랜딩
- **추가 도구** - 기존 CRM, 티켓팅 또는 지식 기반 시스템과 통합
- **다국어 지원** - 글로벌 고객을 위한 추가 국제화
- **고급 분석** - 지원 팀 인사이트를 위한 사용자 정의 대시보드 구현
- **모바일 최적화** - 모바일 디바이스에서 인터페이스가 잘 작동하도록 보장

### 정리

이 워크샵에서 생성된 리소스를 정리할 준비가 되면:

**정리할 준비가 되셨나요?** [랩 6: 정리로 진행 →](lab-06-cleanup.ipynb)

---

**🎊 Amazon Bedrock AgentCore 엔드투엔드 워크샵 완료를 축하합니다!**

Amazon Bedrock AgentCore 기능을 사용하여 프로토타입부터 고객 대면 애플리케이션까지 완전한 프로덕션 준비 AI 에이전트 솔루션을 성공적으로 구축했습니다.